In [ ]:
%load_ext autoreload
%autoreload 2
from colocate import ui, erddap_query, run, plot

#import holoviews as hv
#hv.extension(case_sensitive_completion=True)

import hvplot
import hvplot.pandas

In [ ]:
ui.m

In [ ]:
ui.dates

In [ ]:
ui.drpdwn

In [ ]:
ui.kw

In [ ]:
ui.button

In [ ]:
ui.df

In [ ]:
all_coords = erddap_query.get_coordinates(ui.df, **ui.kw)

In [ ]:
all_coords

In [ ]:
all_coords.columns

In [ ]:
figure = plot.plot_datashader(all_coords)

In [ ]:
figure

In [ ]:
print(figure)

In [ ]:
import panel as pn
import param
pn.extension()
pn.__version__

In [ ]:
# list type is necessary to add an empty value, otherwise could use numpy.ndarrays directly in ObjectSelector:
server_list = ui.df['server'].unique().tolist()
server_list[:0] = [None]
institution_list = ui.df['Institution'].unique().tolist()
institution_list[:0] = [None]
dataset_id_list = servers = ui.df['Dataset ID'].unique().tolist()
dataset_id_list[:0] = [None]

class ErddapParams(param.Parameterized):
    #sel_server = param.ObjectSelector(objects=servers)
    #sel_institution = param.ObjectSelector(objects=institutions)
    #sel_datatypes = param.ObjectSelector(objects=datatypes)
    
    sel_server = param.ObjectSelector(objects=server_list, default=None, allow_None=True, label="Select an ERDDAP Server:")
    sel_institution = param.ObjectSelector(objects=institution_list, default=None, allow_None=True, label="Select an Institution:")
    #sel_datatypes = param.ObjectSelector(objects=datatypes)

    #sel_server = param.ObjectSelector(objects=servers, default="", check_on_set=False, allow_None=True, label="Select an ERDDAP Server:")
    #sel_institution = param.ObjectSelector(objects=instiutions, default="", check_on_set=False, allow_None=True, label="Select an Institution:")

#erddap_params = ErddapParams(servers=servers.tolist(), institutions=institutions.tolist())
erddap_params = ErddapParams()

In [ ]:
#filtered_ids_list = filtered_df['Dataset ID'].unique().tolist()
#filtered_ids_list[:0] = [None]

class ChartParams(param.Parameterized):
    sel_datasetid = param.ObjectSelector(objects=filtered_ids_list, default=None, allow_None=True, label="Select Dataset to Plot:")
    
chart_params = ChartParams()

In [ ]:
filtered_df = ui.df

@pn.depends(erddap_params.param.sel_server, erddap_params.param.sel_institution, chart_params.param.sel_datasetid)
def filter_df(server, institution, datsetid):
    
    if server is not None:
        try:
            filter &= ui.df.server.str.contains(server)
        except NameError:
            filter = ui.df.server.str.contains(server) 
            
    if institution is not None:
        try:
            filter &= ui.df.Institution.str.contains(institution)
        except NameError:
            filter = ui.df.Institution.str.contains(institution)
    
    #filter &=  ui.df.server=server if server is not None
    #filter &=  ui.df.institution=institution if institution is not None
    try:
        
        # below attempts to dynamically update the Params pulldown classes based on the updated DataFrame results, but doesn't work, yet...
        server_list_filter = ui.df[filter].server.unique().tolist()
        server_list_filter[:0] = [None]
        #erddap_params.param.sel_server = param.ObjectSelector(objects=server_list_filter, default=None, allow_None=True, label="Select an ERDDAP Server:")
        #ErddapParams.param.sel_server = param.ObjectSelector(objects=server_list_filter, default=None, allow_None=True, label="Select an ERDDAP Server:")
        #server.objects = server_list_filter
        
        institution_list_filter = ui.df[filter].Institution.unique().tolist()
        institution_list_filter[:0] = [None]
        #erddap_params.param.sel_institution = param.ObjectSelector(objects=institution_list_filter, default=None, allow_None=True, label="Select an Institution:")
        #ErddapParams.param.sel_institution = param.ObjectSelector(objects=institution_list_filter, default=None, allow_None=True, label="Select an Institution:")
        #institution.objects = institution_list_filter
        
        #id_list_filter = ui.df[filter]['Dataset ID'].unique().tolist()
        #datsetid.objects = id_list_filter
        
        filtered_df = ui.df[filter].drop(columns=['Summary']).head(10)
        return ui.df[filter].drop(columns=['Summary']).head(10)
    except Exception: 
        return ui.df.drop(columns=['Summary']).head(10)

In [ ]:
#gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=1000)
gspec = pn.GridSpec(sizing_mode='stretch_both', height_policy='fit', width_policy='auto', height=1600)
gspec[0, 0] = erddap_params
gspec[0, 1:4] = pn.panel(filter_df, width=800, max_height=300)
gspec[1, 0] = chart_params
gspec[2, :] = pn.Spacer(background='purple', margin=0)
gspec